In [1]:
from bs4 import BeautifulSoup

path = 'produced_data/header.xml'

with open(path, 'r', encoding='utf-8') as f:
    def retrieve_person(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        persons = soup.find_all('person')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [(person['xml:id'], person.find('persName').text) for person in persons]

        return xml_ids_and_value_labels
    
    def retrieve_places(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        places = soup.find_all('place')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [{place.find('placeName').text: place['xml:id']} for place in places]

        return xml_ids_and_value_labels

# Example usage:

    xml_ids_and_value_labels = retrieve_places(f)

    print(xml_ids_and_value_labels)

[{'Urbino': 'Urbino'}, {'Europa': 'Europe'}, {'Genova': 'Genoa'}, {'Nemours': 'Nemours'}, {'Grecia': 'Greece'}, {'Venezia': 'Venice'}, {'Napoli': 'Naples'}, {'Bologna': 'Bologna'}, {'Museo Civico Medievale di Bologna': 'MCMBologna'}, {'Uffizi': 'Uffizi'}, {'basilica di San Petronio': 'BSanPetronio'}, {'Egitto': 'Egypt'}, {'Palazzo Bolognini Amorini': 'palazzo-salina-amorini'}, {'Italia': 'Italy'}, {'Cremona': 'Cremona'}, {'National Gallery': 'NGLondon'}, {'Roma': 'Rome'}, {'Musei Vaticani': 'VaticanMuseums'}, {'Madrid': 'Madrid'}, {'Museo del Prado': 'PradoMuseum'}, {'Palermo': 'Palermo'}, {'Sicilia': 'Sicily'}, {'Isola di Capri': 'Capri'}, {'monastero della Santissima Annunziata\n': 'Annunziata-Paternò'}, {'Imola': 'Imola'}, {'Parigi': 'Paris'}, {'Francia': 'France'}, {'Pisa': 'Pisa'}, {'Firenze': 'Florence'}, {'Hampton Court': 'HCourt'}, {'chiesa di San Lorenzo in Fonte': 'sanlinfonte'}, {'Accademia nazionale di San Luca': 'accademia-san-luca-roma'}, {'basilica di San Domenico': 'san

In [9]:
from realNameList import personList
from placeList import placeList
from dictworklocation import worklocationdict
import sparql_dataframe
import pandas

#store the people missing from wikidata and the multiple entries
missingpeople=[]
multiplepeople=[]

placeListLabel = list(key for dictionary in placeList for key in dictionary)       

def createrdf(slicer1, slicer2):
       result = []        
       for x,y in personList[slicer1:slicer2]:
              #result=""
              name = y

              endpoint = 'https://query.wikidata.org/sparql'
              
              queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "%s"@it.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "it")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

              df = sparql_dataframe.get(endpoint, queryPerson, post='True')



              #check if there's only one person in wikidata
              
              same = df['human'].unique()
              
              if df.empty:
                     endpoint = 'https://query.wikidata.org/sparql'
              
                     queryPerson = """
              SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel ?workLocation ?workLocationLabel

              WHERE {
              ?human wdt:P31 wd:Q5;  # Instances of human (Q5)
                     rdfs:label "%s"@en.  # Label in Italian
              ?human rdfs:label ?humanLabel.  # Get the label
              FILTER(LANG(?humanLabel) = "en")  # Filter for Italian labels

              OPTIONAL {?human wdt:P21 ?gender.
                     ?gender rdfs:label ?genderLabel.
                     FILTER(LANG(?genderLabel) = "it")}
              OPTIONAL{?human wdt:P569 ?birthDate.}
              OPTIONAL{?human wdt:P19 ?birthPlace.
                     ?birthPlace rdfs:label ?birthPlaceLabel.
                     FILTER(LANG(?birthPlaceLabel) = "it")}
              OPTIONAL{?human wdt:P570 ?deathDate.}
              OPTIONAL{?human wdt:P20 ?deathPlace.
                     ?deathPlace rdfs:label ?deathPlaceLabel.
                     FILTER(LANG(?deathPlaceLabel) = "it") }
              OPTIONAL{?human wdt:P106 ?occupation.
                     ?occupation rdfs:label ?occupationLabel.
                     FILTER(LANG(?occupationLabel) = "it")}
              OPTIONAL {?human wdt:P937 ?workLocation.
                     ?workLocation rdfs:label ?workLocationLabel.
                     FILTER(LANG(?workLocationLabel) = "it")}

              }
              """%(name)

                     df = sparql_dataframe.get(endpoint, queryPerson, post='True')

                     if df.empty:

                            missingpeople.append((x,y))
              
              elif len(same)>1:
                     multiple_entries= same.tolist()
                     multiplepeople.append(multiple_entries)


              else:
                     birthPlaceLabel = df['birthPlaceLabel'][0] 
                     deathPlaceLabel = df['deathPlaceLabel'][0]
                     createrdf = """
<rdf:Description tei:ref='{id}' rdf:about='{wikidataentity}'>
       <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
       <rdf:label xml:lan="it">{label}</rdf:label>
       <crm:P61_has_gender>{genderLabel}</crm:P61_has_gender>
       <schema:birthDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{birthDate}</schema:birthDate>
       <schema:deathDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">{deathDate}</schema:deathDate>\n""".format(id=x, wikidataentity=df['human'][0], label=df['humanLabel'][0], genderLabel=df['genderLabel'][0], birthDate=df['birthDate'][0], deathDate=df['deathDate'][0])
                     
              #add birth place as referenced in the header or as entity in wikidata
              
                     if birthPlaceLabel in placeList:
                     
                            birthPlaceLabelId= placeList[birthPlaceLabel]

              # #ho dei dubbi sul primo caso

                            createrdf = createrdf + '<schema:birthPlace xml:ref="%s">%s</schema:birthPlace>\n'%(birthPlaceLabelId, birthPlaceLabel)

                     else:
                            createrdf = createrdf + '<schema:birthPlace rdf:resource="%s">%s<schema:birthPlace>\n'%(df['birthPlace'][0], df['birthPlaceLabel'][0])

              # #add death place as referenced in the header or as entity in wikidata

                     if deathPlaceLabel in placeList:

                            deathPlaceLabelId= placeList[deathPlaceLabel]

              # #ho dei dubbi sul primo caso
                            createrdf = createrdf + '\t<schema:deathPlace rdf:resource="%s">%s</schema:deathPlace>\n'%(deathPlaceLabelId, deathPlaceLabel)

                     else:
                            createrdf = createrdf + '\t<schema:deathPlace rdf:resource="%s">%s</schema:deathPlace>\n'%(df['deathPlace'][0], df['deathPlaceLabel'][0])

                     if df['occupation'][0] == 'Nan':
                            for occupation in df['occupation'].unique():
                                   occupationLabel = df.loc[df['occupation'] == occupation, 'occupationLabel'].values[0]
                                   createrdf = createrdf + '<schema:hasOccupation rdf:resource="%s">%s</schema:hasOccupation>\n'%(occupation, occupationLabel)
                     
                     if df['humanLabel'][0] in worklocationdict.keys():
                            for i in worklocationdict[df['humanLabel'][0]]:
                                   if i in placeListLabel:
                                          if i in df['workLocationLabel'].unique():

                                                 worklocationlabel = df.loc[df['workLocationLabel'] == i, 'workLocation'].values[0]
                                                 createrdf = createrdf + '\t<schema:workLocation rdf:resource"%s">%s</schema:workLocation>\n'%(worklocationlabel, i) 
                            
                     createrdf = createrdf + "</rdf:Description>"
                     result.append(createrdf)

       print(missingpeople, multiplepeople)
       return '\n'.join(result)

test = createrdf(0,10)
print(test)

[('FdiGiorgioMartini', 'Federico di Giorgio Martini'), ('LdiCanossa', 'Ludovico da Canossa')] [['http://www.wikidata.org/entity/Q453113', 'http://www.wikidata.org/entity/Q18814526', 'http://www.wikidata.org/entity/Q18814525'], ['http://www.wikidata.org/entity/Q860949', 'http://www.wikidata.org/entity/Q60711238', 'http://www.wikidata.org/entity/Q63870953'], ['http://www.wikidata.org/entity/Q3887170', 'http://www.wikidata.org/entity/Q82925573']]

<rdf:Description tei:ref='GdaMontefeltro' rdf:about='http://www.wikidata.org/entity/Q969797'>
       <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
       <rdf:label xml:lan="it">Guidobaldo da Montefeltro</rdf:label>
       <crm:P61_has_gender>maschio</crm:P61_has_gender>
       <schema:birthDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">1472-01-26T00:00:00Z</schema:birthDate>
       <schema:deathDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date">1508-04-20T00:00:00Z</schema:deathDate>
<schema:birthPlace rdf:resour

In [3]:
#query for places
for i in placeList[:1]:

       label = list(i.items())
       placelabel = label[0][0]
       placelabelid = label[0][1]
       

       endpoint = 'https://query.wikidata.org/sparql'

       queryPlace= """SELECT ?place ?placeLabel ?coordinate  WHERE {
       ?place wdt:P31/wdt:P279* wd:Q2221906; # Instances of place (Q2221906) or its subclasses
              rdfs:label "%s"@it; # Label in English
              wdt:P625 ?coordinate.
       ?place rdfs:label ?placeLabel. # Get the label
       FILTER(LANG(?placeLabel) = "en") # Filter for English labels
       }
       """%placelabel

       df = sparql_dataframe.get(endpoint, queryPlace, post='True')

       coordinate = df['coordinate'][0]
       cleancoordinate = coordinate[6:-1]
       finalcoordinate = cleancoordinate.split(' ')

       createrdf ="""
<rdf:Description tei:ref="{id}" rdf:about="{about}">
       <rdf:type rdf:resource="{wid}"/>
       <rdfs:label xml:lang="en">{label}</rdfs:label>
       <geo:lat>{lat}</geo:lat>
       <geo:long>{long}</geo:long>
</rdf:Description>

       """.format(id=placelabelid, wid = dfplace['place'][0], label = placelabel, lat = finalcoordinate[1], long = finalcoordinate[0], about = df['place'][0])


print(createrdf)

NameError: name 'dfplace' is not defined